In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_plotly, plot_components_plotly

In [2]:
data = pd.read_csv('../data/train.csv')
data.head()

,Date,store,product,number_sold
0,2010-01-01,0,0,801
1,2010-01-02,0,0,810
2,2010-01-03,0,0,818
3,2010-01-04,0,0,796
4,2010-01-05,0,0,808


In [3]:
prod_0 = data.loc[(data['product']==0) & (data['store']==0)].copy()
prod_0

,Date,store,product,number_sold
0,2010-01-01,0,0,801
1,2010-01-02,0,0,810
2,2010-01-03,0,0,818
3,2010-01-04,0,0,796
4,2010-01-05,0,0,808
...,...,...,...,...
3282,2018-12-27,0,0,847
3283,2018-12-28,0,0,854
3284,2018-12-29,0,0,839
3285,2018-12-30,0,0,847


In [4]:
prod_0.drop(columns=['product', 'store'], inplace=True)
prod_0.head(3)

,Date,number_sold
0,2010-01-01,801
1,2010-01-02,810
2,2010-01-03,818


In [5]:
prod_0.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3287 entries, 0 to 3286
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         3287 non-null   object
 1   number_sold  3287 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 77.0+ KB


In [6]:
prod_0['Date'] = pd.to_datetime(prod_0['Date'])
#prod_0.set_index('Date',inplace=True)
prod_0.head(3)

,Date,number_sold
0,2010-01-01,801
1,2010-01-02,810
2,2010-01-03,818


In [7]:
prod_0.tail()

,Date,number_sold
3282,2018-12-27,847
3283,2018-12-28,854
3284,2018-12-29,839
3285,2018-12-30,847
3286,2018-12-31,839


In [8]:
prod_0.rename(columns={'Date':'ds', 'number_sold':'y'}, inplace=True)
prod_0.head(3)

,ds,y
0,2010-01-01,801
1,2010-01-02,810
2,2010-01-03,818


## Modeling

In [9]:
param_grid = {
    'seasonality_mode': ['additive', 'multiplicative'],
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5, 1],
    'seasonality_prior_scale': [0.1, 1.0, 5.0, 10.0]
}

all_results = []

for cps in param_grid['changepoint_prior_scale']:
    for sps in param_grid['seasonality_prior_scale']:
        for mode in param_grid['seasonality_mode']:
                     m = Prophet(
    seasonality_mode=mode,
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    changepoint_prior_scale=cps,
    seasonality_prior_scale=sps,
    interval_width=0.90
)
m.fit(prod_0)

cv_results = cross_validation(m, initial='450 days', period='30 days', horizon='90 days')
performance = performance_metrics(cv_results)
performance['cps'] = cps
performance['sps'] = sps
performance['mode'] = mode
all_results.append(performance)

results_df = pd.concat(all_results)
print(results_df.groupby(['cps', 'sps', 'mode'])['rmse'].mean().sort_values())

16:36:34 - cmdstanpy - INFO - Chain [1] start processing
16:36:38 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/92 [00:00<?, ?it/s]

16:36:38 - cmdstanpy - INFO - Chain [1] start processing
16:36:38 - cmdstanpy - INFO - Chain [1] done processing
16:36:39 - cmdstanpy - INFO - Chain [1] start processing
16:36:39 - cmdstanpy - INFO - Chain [1] done processing
16:36:40 - cmdstanpy - INFO - Chain [1] start processing
16:36:40 - cmdstanpy - INFO - Chain [1] done processing
16:36:41 - cmdstanpy - INFO - Chain [1] start processing
16:36:41 - cmdstanpy - INFO - Chain [1] done processing
16:36:41 - cmdstanpy - INFO - Chain [1] start processing
16:36:41 - cmdstanpy - INFO - Chain [1] done processing
16:36:42 - cmdstanpy - INFO - Chain [1] start processing
16:36:42 - cmdstanpy - INFO - Chain [1] done processing
16:36:43 - cmdstanpy - INFO - Chain [1] start processing
16:36:43 - cmdstanpy - INFO - Chain [1] done processing
16:36:44 - cmdstanpy - INFO - Chain [1] start processing
16:36:45 - cmdstanpy - INFO - Chain [1] done processing
16:36:45 - cmdstanpy - INFO - Chain [1] start processing
16:36:45 - cmdstanpy - INFO - Chain [1]

cps  sps   mode          
1    10.0  multiplicative    11.414913
Name: rmse, dtype: float64


In [10]:
 m = Prophet(
    seasonality_mode='multiplicative',
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    changepoint_prior_scale=1,
    seasonality_prior_scale=10,
    interval_width=0.90
)
m.fit(prod_0)

16:37:57 - cmdstanpy - INFO - Chain [1] start processing
16:37:58 - cmdstanpy - INFO - Chain [1] done processing


In [11]:
future = m.make_future_dataframe(periods=365, freq='D')
forecast = m.predict(future)
forecast.tail()


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
3647,2019-12-27,847.028612,818.445414,868.784917,825.765945,867.303191,-0.004841,-0.004841,-0.004841,-0.000541,-0.000541,-0.000541,-0.004300,-0.004300,-0.004300,0.0,0.0,0.0,842.928058
3648,2019-12-28,847.044933,819.082799,869.685083,825.644298,867.396055,-0.003572,-0.003572,-0.003572,-0.000456,-0.000456,-0.000456,-0.003117,-0.003117,-0.003117,0.0,0.0,0.0,844.018882
3649,2019-12-29,847.061254,819.261460,871.339979,825.524266,867.488919,-0.002194,-0.002194,-0.002194,-0.000240,-0.000240,-0.000240,-0.001954,-0.001954,-0.001954,0.0,0.0,0.0,845.202499
3650,2019-12-30,847.077575,820.584008,872.296079,825.404234,867.581783,-0.000962,-0.000962,-0.000962,-0.000149,-0.000149,-0.000149,-0.000813,-0.000813,-0.000813,0.0,0.0,0.0,846.262471
3651,2019-12-31,847.093896,821.440860,874.761159,825.284201,867.674647,0.001337,0.001337,0.001337,0.001034,0.001034,0.001034,0.000304,0.000304,0.000304,0.0,0.0,0.0,848.226808


In [12]:
fig_forecast = plot_plotly(m, forecast)
fig_forecast.show()

In [13]:
forecast_out = forecast[['ds', 'yhat']]
forecast_out.rename(columns={'ds':'Date', 'yhat':'number_sold'}, inplace=True)
forecast_out.head()

C:\Users\User\AppData\Local\Temp\ipykernel_26676\3798538108.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Date,number_sold
0,2010-01-01,809.770341
1,2010-01-02,810.751932
2,2010-01-03,811.820246
3,2010-01-04,812.771277
4,2010-01-05,814.589210


In [14]:
forecast_out.tail()

,Date,number_sold
3647,2019-12-27,842.928058
3648,2019-12-28,844.018882
3649,2019-12-29,845.202499
3650,2019-12-30,846.262471
3651,2019-12-31,848.226808


In [18]:
#get forecast for 2019 only
forecast_2019 = forecast_out.loc[forecast_out['Date'].dt.year == 2019]
forecast_2019

,Date,number_sold
3287,2019-01-01,843.424463
3288,2019-01-02,843.794891
3289,2019-01-03,844.324945
3290,2019-01-04,844.775623
3291,2019-01-05,845.704267
...,...,...
3647,2019-12-27,842.928058
3648,2019-12-28,844.018882
3649,2019-12-29,845.202499
3650,2019-12-30,846.262471


## Evaluation


In [17]:
test = pd.read_csv('../data/test data.csv')
test = test.loc[(test['product']==0)&(test['store']==0)]
test.drop(columns=['product', 'store'], inplace=True)
test

,Date,number_sold
0,2019-01-01,845
1,2019-01-02,851
2,2019-01-03,840
3,2019-01-04,842
4,2019-01-05,845
...,...,...
360,2019-12-27,848
361,2019-12-28,856
362,2019-12-29,855
363,2019-12-30,862


In [16]:
cv_results = cross_validation(m, initial='450 days', period='30 days', horizon='90 days')
performance = performance_metrics(cv_results)
performance

  0%|          | 0/92 [00:00<?, ?it/s]

16:38:01 - cmdstanpy - INFO - Chain [1] start processing
16:38:01 - cmdstanpy - INFO - Chain [1] done processing
16:38:01 - cmdstanpy - INFO - Chain [1] start processing
16:38:01 - cmdstanpy - INFO - Chain [1] done processing
16:38:02 - cmdstanpy - INFO - Chain [1] start processing
16:38:02 - cmdstanpy - INFO - Chain [1] done processing
16:38:02 - cmdstanpy - INFO - Chain [1] start processing
16:38:02 - cmdstanpy - INFO - Chain [1] done processing
16:38:02 - cmdstanpy - INFO - Chain [1] start processing
16:38:03 - cmdstanpy - INFO - Chain [1] done processing
16:38:03 - cmdstanpy - INFO - Chain [1] start processing
16:38:03 - cmdstanpy - INFO - Chain [1] done processing
16:38:03 - cmdstanpy - INFO - Chain [1] start processing
16:38:03 - cmdstanpy - INFO - Chain [1] done processing
16:38:04 - cmdstanpy - INFO - Chain [1] start processing
16:38:04 - cmdstanpy - INFO - Chain [1] done processing
16:38:04 - cmdstanpy - INFO - Chain [1] start processing
16:38:04 - cmdstanpy - INFO - Chain [1]

,horizon,mse,rmse,mae,mape,mdape,smape,coverage
0,9 days,105.004332,10.247162,8.162940,0.009857,0.008484,0.009856,0.876812
1,10 days,104.450290,10.220092,8.140437,0.009828,0.008307,0.009828,0.874396
2,11 days,103.171123,10.157319,8.136121,0.009829,0.008529,0.009832,0.876812
3,12 days,105.500967,10.271366,8.187446,0.009891,0.008529,0.009891,0.873188
4,13 days,102.068684,10.102905,8.030570,0.009714,0.008217,0.009712,0.886473
...,...,...,...,...,...,...,...,...
77,86 days,170.141022,13.043812,9.916557,0.011958,0.009692,0.011954,0.869565
78,87 days,171.937505,13.112494,9.931182,0.011968,0.009692,0.011962,0.867150
79,88 days,170.753286,13.067260,9.836880,0.011876,0.009361,0.011865,0.869565
80,89 days,164.532620,12.827027,9.621890,0.011608,0.009061,0.011599,0.887681


In [19]:
from statsmodels.tools.eval_measures import rmse
print('RMSE:', rmse(test['number_sold'], forecast_2019['number_sold']))

RMSE: 12.031729014373262
